In [38]:
import os
if(not os.getcwd().endswith("crchum")):
    os.chdir("..")
os.getcwd()

'/Users/antoinemagron/Documents/EPFL/PDM/crchum'

In [39]:
with open("./docs/benchmarks/mcq/raw.txt", 'r') as f:
    raw = f.read()

In [40]:
raw

'Following are the preferred responses for the Multiple-Choice Questions in this CONTINUUM issue. The questions and answer options are repeated, and the preferred response appears in bold print followed by an explanation and a reference with which you may seek more specific information. You are encouraged to review the responses and explanations carefully to evaluate your general understanding of the course material. The comments and references included with each question are intended to encourage independent study.\n\n    ►1. Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?\n        A. appropriately weaning antiepileptic drugs\n        B. describing events in ambiguous terms\n        C. ensuring that patients continue to refrain from driving a motor vehicle\n        D. ordering follow-up EEG in 6 to 12 months\n        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant\n\nThe correct answer is

In [41]:
SEP = "►"
len(raw.split(SEP)[1:]), "questions"

(40, 'questions')

In [52]:
trimed = [[_ for _ in question.split("\n") if _.strip() != ''] for question in raw.split(SEP)[1:]]
processed = []
for question in trimed:
    complete = {}
    q = "\n".join(question[:-1])
    a = question[-1]
    complete["answer"] = a.replace("The correct answer is ", "")[0]
    complete["answer_full"] = a
    complete["index"], complete["question"] = q.split(". ")[0], ". ".join(q.split(". ")[1:])
    processed.append(complete)

In [53]:
processed

[{'answer': 'A',
  'answer_full': 'The correct answer is A (appropriately weaning antiepileptic drugs). After a diagnosis of psychogenic nonepileptic attacks, one of the major roles of the neurologist is the rational weaning of antiepileptic medications. This should be done with the caveat that coexistent epilepsy may also be present, so the physician needs to keep an open mind regarding new events (although routine surveillance EEGs are not necessarily indicated). Driving restrictions should be handled on an individual case basis, although most patients can return to driving with little chance of injury. Neurologists should use unambiguous terms when describing the diagnosis to the patient and other providers so that the misdiagnosis of epilepsy is not perpetuated. While mental health care is often needed, the routine prescription of antidepressants is not always indicated. For more information, refer to page 724 of the CONTINUUM article “Nonepileptic Behavioral Disorders: Diagnosis a

In [54]:
import json
with open("docs/benchmarks/mcq/processed.json", "w") as f:
    json.dump(processed, f)

In [40]:
OFFSET_A = 65
letters = {i - OFFSET_A: chr(i) for i in range(OFFSET_A, 65+26)}
letters

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z'}

### Self assessment 

In [18]:
import pandas as pd


selfassessment = pd.read_excel("docs/benchmarks/self_assessment/AES7-8.xlsx").T
cols = selfassessment.loc["Numéro"].values
selfassessment = selfassessment.drop("Numéro")
selfassessment.columns = cols
selfassessment = selfassessment.reset_index()

In [21]:
aes8, aes7 = selfassessment.iloc[:100], selfassessment.iloc[100:]

In [37]:
print(aes7["Question"].values[0])

Based on the ILAE consensus classification, which of the following findings are seen in the most common histopathological subtype of hippocampal sclerosis?
A. Neuronal cell loss in the CA2 and CA3 regions
B. Neuronal cell loss in the CA1 and CA3 regions
C. Neuronal cell loss in the CA1 and CA4 regions
D. Neuronal cell loss in the dentate regions


In [32]:
print(aes7["Input "].values[1])

You will be given a multiple choice question, please select the correct answer :  "What was the median seizure reduction 9 years after implantaiton in long term studies of safety and efficacy for brain-responsive neurostimulation (RNS)?
A. 55%
B. 63%
C. 75%
D. 90%"


In [58]:
aes8 = aes8.drop("Is_image", axis=1)
aes7 = aes7.drop("Is_image", axis=1)
columns = ["index", "question", "answer", "human_accuracy", "difficulty", "topic", "answer_full", "ref.", "input"]
aes7.columns = columns
aes8.columns = columns

In [62]:
for t, w in [("aes7", aes7), ("ae8", aes8)]:
    with open(f"docs/benchmarks/self_assessment/{t}_processed.json", "w") as f:
        json.dump(w.to_dict("records"), f)